In [13]:
import pandas as pd 
import requests 
from secrets_config import api_key
import jinja2 as j2

Extract data

In [14]:
# read list of cities
df_cities = pd.read_csv("data/australian_capital_cities.csv")
df_cities.head()

,city_name
0,canberra
1,sydney
2,darwin
3,brisbane
4,adelaide


In [15]:
# request data for each city (json) and push to a list 
weather_data = []
for city_name in df_cities["city_name"]:
    params = {
        "q": city_name,
        "units": "metric",
        "appid": api_key
    }
    response = requests.get(f"http://api.openweathermap.org/data/2.5/weather", params=params)
    if response.status_code == 200: 
        weather_data.append(response.json())
    else: 
        raise Exception("Extracting weather api data failed. Please check if API limits have been reached.")

In [16]:
# read data into a dataframe to create a consistent schema 
df = pd.json_normalize(weather_data, max_level=0)
df.head()

,coord,weather,base,main,visibility,wind,clouds,dt,sys,timezone,id,name,cod
0,"{'lon': 149.1281, 'lat': -35.2835}","[{'id': 804, 'main': 'Clouds', 'description': ...",stations,"{'temp': 7.88, 'feels_like': 5.92, 'temp_min':...",10000,"{'speed': 3.07, 'deg': 105, 'gust': 6.51}",{'all': 88},1663058559,"{'type': 2, 'id': 2006597, 'country': 'AU', 's...",36000,2172517,Canberra,200
1,"{'lon': 151.2073, 'lat': -33.8679}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 13.25, 'feels_like': 12.3, 'temp_min'...",10000,"{'speed': 6.17, 'deg': 170}",{'all': 40},1663058252,"{'type': 2, 'id': 2002865, 'country': 'AU', 's...",36000,2147714,Sydney,200
2,"{'lon': 130.8418, 'lat': -12.4611}","[{'id': 801, 'main': 'Clouds', 'description': ...",stations,"{'temp': 31.75, 'feels_like': 34.14, 'temp_min...",10000,"{'speed': 5.14, 'deg': 20}",{'all': 20},1663058428,"{'type': 1, 'id': 9574, 'country': 'AU', 'sunr...",34200,2073124,Darwin,200
3,"{'lon': 153.0281, 'lat': -27.4679}","[{'id': 801, 'main': 'Clouds', 'description': ...",stations,"{'temp': 18.01, 'feels_like': 17.8, 'temp_min'...",10000,"{'speed': 5.14, 'deg': 130}",{'all': 20},1663058312,"{'type': 2, 'id': 2005393, 'country': 'AU', 's...",36000,2174003,Brisbane,200
4,"{'lon': 138.6, 'lat': -34.9333}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 19.71, 'feels_like': 19.09, 'temp_min...",10000,"{'speed': 7.2, 'deg': 50}",{'all': 0},1663058242,"{'type': 1, 'id': 9566, 'country': 'AU', 'sunr...",34200,2078025,Adelaide,200


Load data

In [17]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Float, JSON # https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_creating_table.htm
from sqlalchemy.engine import URL
from sqlalchemy.dialects import postgresql
from secrets_config import db_user, db_password, db_server_name, db_database_name

In [18]:
# create connection to database 
connection_url = URL.create(
    drivername = "postgresql+pg8000", 
    username = db_user,
    password = db_password,
    host = db_server_name, 
    port = 5432,
    database = db_database_name, 
)

engine = create_engine(connection_url)

In [19]:
# specify target table schema 
raw_table = "raw_weather"

meta = MetaData()
weather_table = Table(
    raw_table, meta, 
    Column("dt", Integer, primary_key=True),
    Column("id", Integer, primary_key=True),
    Column("coord", JSON),
    Column("weather", JSON),
    Column("base", String),
    Column("main", JSON),
    Column("visibility", Integer),
    Column("rain", String),
    Column("wind", JSON),
    Column("clouds", JSON),
    Column("sys", JSON),
    Column("timezone", Integer),
    Column("name", String),
    Column("cod", Integer)
)
meta.create_all(engine) # creates table if it does not exist 

In [20]:
insert_statement = postgresql.insert(weather_table).values(df.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['id', 'dt'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['id', 'dt']})
engine.execute(upsert_statement)

Transform data (SQL)

In [21]:
target_table = "staging_weather"

with open(f"models/{target_table}.sql") as f: 
    raw_sql = f.read()
print(raw_sql)

drop table if exists {{ target_table }}; 

-- create table as 
create table {{ target_table }} as (
    select 
        to_timestamp(dt) as datetime, 
        id, 
        name, 
        cast(coord ->> 'lon' as numeric) as lon,
        cast(coord ->> 'lat' as numeric) as lat,
        weather -> 0 ->> 'main' as weather_main,
        weather -> 0 ->> 'description' as weather_description,
        base, 
        cast(main ->> 'temp' as numeric) as temperature,
        cast(main ->> 'pressure' as numeric) as pressure,
        cast(main ->> 'humidity' as numeric) as humidity,
        cast(clouds ->> 'all' as numeric) as clouds
    from raw_weather
); 



In [22]:
parsed_sql = j2.Template(raw_sql).render(target_table = target_table)

In [23]:
engine.execute(parsed_sql)